In [1]:
import contextlib
import os

import pandas as pd
from constants import max_steps, replicates

from mesa.batchrunner import batch_run_not_stupid
from mesa.examples.advanced.epstein_civil_violence.model import EpsteinCivilViolence

In [2]:
files = [f"data_{x:04}.csv" for x in range(1)]
files

['data_0000.csv']

In [3]:
def in_file(filename: str):
    return f"inputs/{filename}"


def out_file(filename: str):
    return f"outputs/{filename}"

In [4]:
def run_for_file(filename: str):
    data = pd.read_csv(in_file(filename), index_col=0)
    print(f"Loaded {len(data)} parameter samples.")

    result = batch_run_not_stupid(
        EpsteinCivilViolence,
        parameters=data,
        iterations=replicates,
        max_steps=max_steps,
        number_processes=None,
        display_progress=True,
    )

    result_df = pd.DataFrame(result)
    with contextlib.suppress(FileExistsError):
        os.mkdir("outputs")
    result_df.to_csv(out_file(filename))

In [5]:
for file in files:
    run_for_file(file)

Loaded 1024 parameter samples.


  0%|          | 0/102400 [00:00<?, ?it/s]

TypeError: EpsteinCivilViolence.__init__() got an unexpected keyword argument 'reversion_rate'